# Conexão Várias tabelas

In [10]:
import dafra as DF
import pandas as pd
import pyodbc

#### Carrega todas as tabelas do diretorio - atenção com as extensoes

In [11]:
diretorio = r"C:\Users\david\Desktop\ProjetosdeCienciasdeDados\Dados\arquivos_excel\meuconsolidado\tratados/"
lista_arquivos = ['Categorias.xlsx','Items.xlsx','Ordens.xlsx','Produtos.xlsx','Clientes.csv']
Categoria = pd.read_excel(diretorio+lista_arquivos[0])
Items = pd.read_excel(diretorio+lista_arquivos[1])
Ordens = pd.read_excel(diretorio+lista_arquivos[2])
Produtos = pd.read_excel(diretorio+lista_arquivos[3])
Clientes = pd.read_csv(diretorio+lista_arquivos[4], delimiter= ',')


### Abre a conexão com o banco de dados

In [12]:
# Variaveis de conexão
driver_ODBC = 'ODBC Driver 17 for SQL Server'
nome_servidor = 'DAVIDFRANCISCO\SQLEXPRESS'
nome_BD = 'AulaPython'
#tabelaSQL = '[Clientes]'
usuario = 'dsds'
senha = '1234'

# Conexão
conexaoDB = pyodbc.connect(f'DRIVER={driver_ODBC};'
                           f'SERVER={nome_servidor};'
                           f'DATABASE={nome_BD};'
                           'Trusted_Connection=yes;')

cursor = conexaoDB.cursor()   # criando cursor de comando

### Truncate - Limpa todas as tabelas do sistema gerenciador banco de dados

In [13]:
cursor.execute(
"""
truncate table [dbo].[Categoria]
truncate table [dbo].[Clientes]
truncate table [dbo].[Items]
truncate table [dbo].[Ordens]
truncate table [dbo].[Produtos]

"""
)   #executa tarefa de  apagar dados
cursor.commit()

### Insert de todas as tabelas tratadas no SQL Server

Carga produto

In [14]:
for index, linha in Produtos.iterrows():  
    cursor.execute("Insert into [Produtos](ID,Nome,Price,Id_Category)values(?,?,?,?)",
                   linha.ID,linha.Nome,linha.Price,linha.Id_Category) 
cursor.commit()   

Carga Categoria

In [15]:
for index, linha in Categoria.iterrows():  
    cursor.execute("Insert into [Categoria](ID,Categoria)values(?,?)",linha.ID,linha.Categoria) 
cursor.commit()

Carga Items

In [16]:
Items['total_price'] = Items['total_price'].astype(float) 
for index, linha in Items.iterrows():
    cursor.execute("Insert into [Items](id,order_id,product_id,quantity,total_price)values(?,?,?,?,?)",
                   linha.id,linha.order_id,linha.product_id,linha.quantity,linha.total_price) 
cursor.commit()

Carga Ordens

In [17]:
for index, linha in Ordens.iterrows():
    cursor.execute("Insert into [Ordens](id,created_at,customer_id,status)values(?,?,?,?)",
                   linha.id,linha.created_at,linha.customer_id,linha.status)   
cursor.commit()  

Carga Clientes

In [18]:
Clientes['created_at'] = pd.to_datetime(Clientes['created_at'])

for index, linha in Clientes.iterrows():   
    linha.email = str(linha.email)  # Converter para o tipo 'str' antes da inserção
    linha.country = str(linha.country)
    linha.state = str(linha.state)
    linha.street = str(linha.street)
    linha.number = str(linha.number)
    linha.additionals = str(linha.additionals)
    
    #linha.cell_phone = str(linha.cell_phone)
    cursor.execute("INSERT INTO [Clientes] (id, created_at,first_name, last_name,email,cell_phone,country, state,street, number, additionals) VALUES (?,?,?,?,?,?,?,?,?,?,?)",linha.id, linha.created_at,linha.first_name,
                   linha.last_name,linha.email,linha.cell_phone,linha.country,linha.state,linha.street,linha.number,linha.additionals)
cursor.commit()

### Fechando a conexão

In [19]:
cursor.close()    #Fechar Cursor
conexaoDB.close() #Fechar Conexao